In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/images-tdl/final_csv.csv
/kaggle/input/samplepesu/Training Dataset_csv/V_HELICOPTER_038_LABELS.csv
/kaggle/input/samplepesu/Training Dataset_csv/V_HELICOPTER_017_LABELS.csv
/kaggle/input/samplepesu/Training Dataset_csv/V_DRONE_025_LABELS.csv
/kaggle/input/samplepesu/Training Dataset_csv/V_DRONE_073_LABELS.csv
/kaggle/input/samplepesu/Training Dataset_csv/V_BIRD_007_LABELS.csv
/kaggle/input/samplepesu/Training Dataset_csv/V_HELICOPTER_012_LABELS.csv
/kaggle/input/samplepesu/Training Dataset_csv/V_AIRPLANE_041_LABELS.csv
/kaggle/input/samplepesu/Training Dataset_csv/V_HELICOPTER_033_LABELS.csv
/kaggle/input/samplepesu/Training Dataset_csv/V_BIRD_001_LABELS.csv
/kaggle/input/samplepesu/Training Dataset_csv/V_DRONE_051_LABELS.csv
/kaggle/input/samplepesu/Training Dataset_csv/V_DRONE_069_LABELS.csv
/kaggle/input/samplepesu/Training Dataset_csv/V_DRONE_043_LABELS.csv
/kaggle/input/samplepesu/Training Dataset_csv/V_DRONE_074_LABELS.csv
/kaggle/input/samplepesu/Training Dataset_c

In [2]:
import cv2
import os
import pandas as pd

# Define the folder path containing the videos
folder_path = '/kaggle/input/samplepesu/Training Dataset_Videos'

# Get a list of all video files in the folder
video_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.mp4')]

# Create a directory to save the extracted frames
if not os.path.exists('frames'):
    os.makedirs('frames')
count = 1
label_list=[]
box_list=[]
name_list=[]
# Loop over all the video files in the folder
for i, video_file in enumerate(video_files[0:30]):
    row_count=0
    print(f"Extracting frames from video file {i+1}/{len(video_files)}: {video_file}")
    # Open the video file
    vidcap = cv2.VideoCapture(video_file)
    csv_name=video_file.split("/")[5].split(".")[0]
    tag=csv_name.split("_")[1]
    csv_name="/kaggle/input/samplepesu/Training Dataset_csv/"+csv_name+"_LABELS.csv"
    df=pd.read_csv(csv_name,header=None,names=["time","AIRPLANE","BIRD","DRONE","HELICOPTER"])
    # Extract frames at every n-th second (e.g., 1 frame per second)
    success = True
    while success:
        filename = f"frames/{count}.jpg"
        success, image = vidcap.read()
        try:
            cv2.imwrite(filename, image)
            if "3x" not in df[tag][row_count] or "4x" not in df[tag][row_count] or ";" not in df[tag][row_count]:
                box_list.append(df[tag][row_count])
                label_list.append(tag)
                name_list.append(f"{count}.jpg")
            count += 1
            row_count+=1                
        except:
            print("EOF")
dic={"name":name_list,"label":label_list,"bounding_box":box_list}
df=pd.DataFrame(dic)
df.to_csv("final_csv.csv",index=False)

Extracting frames from video file 1/280: /kaggle/input/samplepesu/Training Dataset_Videos/V_DRONE_022.mp4
EOF
Extracting frames from video file 2/280: /kaggle/input/samplepesu/Training Dataset_Videos/V_DRONE_011.mp4
EOF
Extracting frames from video file 3/280: /kaggle/input/samplepesu/Training Dataset_Videos/V_DRONE_023.mp4
EOF
Extracting frames from video file 4/280: /kaggle/input/samplepesu/Training Dataset_Videos/V_BIRD_034.mp4
EOF
Extracting frames from video file 5/280: /kaggle/input/samplepesu/Training Dataset_Videos/V_HELICOPTER_041.mp4
EOF
Extracting frames from video file 6/280: /kaggle/input/samplepesu/Training Dataset_Videos/V_DRONE_046.mp4
EOF
Extracting frames from video file 7/280: /kaggle/input/samplepesu/Training Dataset_Videos/V_DRONE_063.mp4
EOF
Extracting frames from video file 8/280: /kaggle/input/samplepesu/Training Dataset_Videos/V_HELICOPTER_050.mp4
EOF
Extracting frames from video file 9/280: /kaggle/input/samplepesu/Training Dataset_Videos/V_BIRD_029.mp4
EOF
Ex

In [3]:
df=pd.DataFrame(dic)
df.to_csv("final_csv.csv",index=False)


In [4]:
s=df['bounding_box'][64062]
d=df['bounding_box'][12520]
print(s,d)
df.drop(df[df['bounding_box']==s].index, inplace = True)
df.drop(df[df['bounding_box']==d].index, inplace = True)

KeyError: 64062

In [ ]:
df = df[~df.apply(lambda row: row.astype(str).str.contains(';').any(), axis=1)]

In [ ]:
df.to_csv("final_csv.csv",index=False)

In [ ]:
df=pd.read_csv("/kaggle/working/final_csv.csv")

In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
import re

from PIL import Image

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torch
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler

from matplotlib import pyplot as plt

# DIR_INPUT = '/kaggle/input/global-wheat-detection'
# DIR_TRAIN = f'{DIR_INPUT}/train'
# DIR_TEST = f'{DIR_INPUT}/test'

In [ ]:
train_df = df
train_df.shape

In [ ]:
def expand_bbox(x):
    z=x.strip("[]").split(",") 
    if len(z)<4:
        r = [-1, -1, -1, -1]
    else:
        z=list(map(float,z))
        r = np.array(z)  
    if len(r) == 0:
        r = [-1, -1, -1, -1]
    return np.array([r])

train_df.loc[:, ['x', 'y', 'w', 'h']] = np.concatenate(train_df['bounding_box'].apply(lambda x: expand_bbox(x)).to_numpy())

In [ ]:
train_df

In [ ]:
image_ids = train_df['name'].unique()
valid_ids = image_ids[-640:]
train_ids = image_ids[:-640]

In [ ]:
valid_df = train_df[train_df['name'].isin(valid_ids)]
train_df = train_df[train_df['name'].isin(train_ids)]

In [ ]:
valid_df.shape, train_df.shape

In [ ]:
class Customise_Data(Dataset):

    def __init__(self, dataframe, image_dir, transforms=None):
        super().__init__()

        self.image_ids = dataframe['name'].unique()
        self.df = dataframe
        self.image_dir = image_dir
        self.transforms = transforms

    def __getitem__(self, index: int):

        image_id = self.image_ids[index]
        records = self.df[self.df['name'] == image_id]
        image = cv2.imread(f'{self.image_dir}/{image_id}', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0

        boxes = records[['x', 'y', 'w', 'h']].values
        boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
        boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
        
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        area = torch.as_tensor(area, dtype=torch.float32)

        # there is only one class
        labels = torch.ones((records.shape[0],), dtype=torch.int64)
        
        # suppose all instances are not crowd
        iscrowd = torch.zeros((records.shape[0],), dtype=torch.int64)
        
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        # target['masks'] = None
        target['image_id'] = torch.tensor([index])
        target['area'] = area
        target['iscrowd'] = iscrowd

        if self.transforms:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': labels
            }
            sample = self.transforms(**sample)
            image = sample['image']
            
            target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)

        return image, target, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]

In [ ]:
# Albumentations
def get_train_transform():
    return A.Compose([
        A.Flip(0.5),
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

def get_valid_transform():
    return A.Compose([
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

In [ ]:
# load a model; pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

In [ ]:
num_classes = 4  # 1 class (wheat) + background

# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [ ]:
class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0


In [ ]:
DIR_TRAIN="/kaggle/working/frames"

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

train_dataset = Custom_Data(train_df,DIR_TRAIN, get_train_transform())
valid_dataset = Custom_Data(valid_df, DIR_TRAIN, get_valid_transform())


# split the dataset in train and test set
indices = torch.randperm(len(train_dataset)).tolist()

train_data_loader = DataLoader(
    train_dataset,
    batch_size=3,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

valid_data_loader = DataLoader(
    valid_dataset,
    batch_size=10,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

In [ ]:
images, targets, image_ids = next(iter(train_data_loader))
images = list(image.to(device) for image in images)
targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

In [ ]:
targets

In [ ]:
boxes = targets[0]['boxes'].cpu().numpy().astype(np.int32)
sample = images[0].permute(1,2,0).cpu().numpy()

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
print(device)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 8))

for box in boxes:
    cv2.rectangle(sample,
                  (box[0], box[1]),
                  (box[2], box[3]),
                  (220, 0, 0), 3)
    
ax.set_axis_off()
ax.imshow(sample)

In [ ]:
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
# lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
lr_scheduler = None

num_epochs = 2

In [ ]:
loss_hist = Averager()
itr = 1

for epoch in range(num_epochs):
    loss_hist.reset()
    
    for images, targets, image_ids in train_data_loader:
        
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()

        loss_hist.send(loss_value)

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if itr % 50 == 0:
            print(f"Iteration #{itr} loss: {loss_value}")

        itr += 1
    
    # update the learning rate
    if lr_scheduler is not None:
        lr_scheduler.step()

    print(f"Epoch #{epoch} loss: {loss_hist.value}") 

In [ ]:
images, targets, image_ids = next(iter(valid_data_loader))

In [ ]:
images

In [ ]:
images = list(img.to(device) for img in images)
targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

In [ ]:
boxes = outputs[1]['boxes'].cpu().numpy().astype(np.int32)
sample = images[1].permute(1,2,0).cpu().numpy()

In [ ]:
model.eval()
cpu_device = torch.device("cpu")

outputs = model(images)
outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]

In [ ]:
sample

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 8))

for box in boxes:
    cv2.rectangle(sample,
                  (box[0], box[1]),
                  (box[2], box[3]),
                  (220, 0, 0), 3)
    
ax.set_axis_off()
ax.imshow(sample)

In [ ]:
torch.save(model.state_dict(), 'fasterrcnn_resnet50_fpn.pth')

In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
import re

from PIL import Image

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torch
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler

from matplotlib import pyplot as plt

# DIR_INPUT = '/kaggle/input/global-wheat-detection'
# DIR_TRAIN = f'{DIR_INPUT}/train'
# DIR_TEST = f'{DIR_INPUT}/test'

WEIGHTS_FILE = '/kaggle/working/fasterrcnn_resnet50_fpn.pth'

In [ ]:
import numpy as np 
import pandas as pd
import tensorflow as tf
import os
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageEnhance
import albumentations as albu
from tqdm.notebook import tqdm

In [ ]:
import cv2
import os
import pandas as pd

# Define the folder path containing the videos
folder_path = '/kaggle/input/samplepesu/Test_data_images'

# Get a list of all video files in the folder
test_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.jpeg')]


names=[]
for x in test_files:
    names.append(x.split("/")[5])

names.sort()
names=[]
for i in range(1,101):
    names.append(str(str(i)+".jpeg"))

names.remove('23.jpeg')
names.remove('27.jpeg')
names.remove('29.jpeg')
names.remove('67.jpeg')
dic={"Test_image":names,"Prediction_String":["" for i in range(0,96)]}
df=pd.DataFrame(dic)
df.to_csv("test_csv.csv",index=False)

In [ ]:
test_df=df

In [ ]:
test_df

In [ ]:
class CustomSetData(Dataset):

    def __init__(self, dataframe, image_dir, transforms=None):
        super().__init__()

        self.image_ids = dataframe['Test_image'].unique()
        self.df = dataframe
        self.image_dir = image_dir
        self.transforms = transforms

    def __getitem__(self, index: int):

        image_id = self.image_ids[index]
        records = self.df[self.df['Test_image'] == image_id]

        image = cv2.imread(f'{self.image_dir}/{image_id}', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0

        if self.transforms:
            sample = {
                'image': image,
            }
            sample = self.transforms(**sample)
            image = sample['image']

        return image, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]

In [ ]:
# Albumentations
def get_test_transform():
    return A.Compose([
        # A.Resize(512, 512),
        ToTensorV2(p=1.0)
    ])

In [ ]:
# load a model; pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, pretrained_backbone=False)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

num_classes = 4 # 1 class (wheat) + background

# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Load the trained weights
model.load_state_dict(torch.load("/kaggle/working/fasterrcnn_resnet50_fpn.pth"))
model.eval()

x = model.to(device)

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

test_dataset = WheatTestDataset(test_df, "/kaggle/input/samplepesu/Test_data_images", get_test_transform())

test_data_loader = DataLoader(
    test_dataset,
    batch_size=2,
    shuffle=False,
    num_workers=4,
    drop_last=False,
    collate_fn=collate_fn
)

In [ ]:
def format_prediction_string(boxes, scores,labels):
    pred_strings = []
    for j in zip(scores, boxes):
        pred_strings.append("{0:.4f} {1} {2} {3} {4} {5}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3],labels[0]))

    return " ".join(pred_strings)

In [ ]:
detection_threshold = 0.5
results = []

for images, image_ids in test_data_loader:

    images = list(image.to(device) for image in images)
    outputs = model(images)


    for i, image in enumerate(images):

        boxes = outputs[i]['boxes'].data.cpu().numpy()
        scores = outputs[i]['scores'].data.cpu().numpy()
        labels=outputs[i]['labels'].data.cpu().numpy()
        
        boxes = boxes[scores >= detection_threshold].astype(np.int32)
        scores = scores[scores >= detection_threshold]
        image_id = image_ids[i]
        
        boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
        boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
        
        result = {
            'image_id': image_id,
            'PredictionString': format_prediction_string(labels,boxes, scores)
        }

        
        results.append(result)

In [ ]:
test_df = pd.DataFrame(results, columns=['image_id', 'PredictionString'])
test_df.head(96)

In [ ]:
print(outputs[1]['labels'].data.cpu().numpy())

In [ ]:
test_df.to_csv("test_csv.csv")

In [ ]:
sample = images[1].permute(1,2,0).cpu().numpy()
boxes = outputs[1]['boxes'].data.cpu().numpy()
scores = outputs[1]['scores'].data.cpu().numpy()

boxes = boxes[scores >= detection_threshold].astype(np.int32)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 8))

for box in boxes:
    cv2.rectangle(sample,
                  (box[0], box[1]),
                  (box[2], box[3]),
                  (220, 0, 0), 2)
    
ax.set_axis_off()
ax.imshow(sample)